In [ ]:
!pip install vtk

In [ ]:
import numpy as np
import torch
from torch import nn, optim
import torch.optim as optim
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
import vtk
from vtk import *
from vtk.util.numpy_support import vtk_to_numpy
import random
import os
import sys
import time

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('Device running:', device)

Device running: cuda


In [ ]:
class SineLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True, is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        # self.enable_dropout = enable_dropout
        # self.dropout_prob = dropout_prob
        self.in_features = in_features
        # if enable_dropout:
        #     if not self.is_first:
        #         self.dropout = nn.Dropout(dropout_prob)
        self.linear = nn.Linear(in_features, out_features, bias=bias)

        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features,
                                             1 / self.in_features)
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0,
                                             np.sqrt(6 / self.in_features) / self.omega_0)


    def forward(self, x):
        x = self.linear(x)
        # if self.enable_dropout:
        #     if not self.is_first:
        #         x = self.dropout(x)
        return torch.sin(self.omega_0 * x)

In [ ]:
class ResidualSineLayer(nn.Module):
    def __init__(self, features, bias=True, ave_first=False, ave_second=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        # self.enable_dropout = enable_dropout
        # self.dropout_prob = dropout_prob
        self.features = features
        # if enable_dropout:
        #     self.dropout_1 = nn.Dropout(dropout_prob)
        self.linear_1 = nn.Linear(features, features, bias=bias)
        self.linear_2 = nn.Linear(features, features, bias=bias)
        self.weight_1 = .5 if ave_first else 1
        self.weight_2 = .5 if ave_second else 1

        self.init_weights()


    def init_weights(self):
        with torch.no_grad():
            self.linear_1.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                           np.sqrt(6 / self.features) / self.omega_0)
            self.linear_2.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                           np.sqrt(6 / self.features) / self.omega_0)

    def forward(self, input):
        linear_1 = self.linear_1(self.weight_1*input)
        # if self.enable_dropout:
        #     linear_1 = self.dropout_1(linear_1)
        sine_1 = torch.sin(self.omega_0 * linear_1)
        sine_2 = torch.sin(self.omega_0 * self.linear_2(sine_1))
        return self.weight_2*(input+sine_2)

In [ ]:
class MyResidualSirenNet(nn.Module):
    def __init__(self, obj):
        super(MyResidualSirenNet, self).__init__()
        # self.enable_dropout = obj['enable_dropout']
        # self.dropout_prob = obj['dropout_prob']
        self.Omega_0=30
        self.n_layers = obj['n_layers']
        self.input_dim = obj['dim']
        self.output_dim = obj['total_vars']
        self.neurons_per_layer = obj['n_neurons']
        self.layers = [self.input_dim]
        for i in range(self.n_layers-1):
            self.layers.append(self.neurons_per_layer)
        self.layers.append(self.output_dim)
        self.net_layers = nn.ModuleList()
        for idx in np.arange(self.n_layers):
            layer_in = self.layers[idx]
            layer_out = self.layers[idx+1]
            ## if not the final layer
            if idx != self.n_layers-1:
                ## if first layer
                if idx==0:
                    self.net_layers.append(SineLayer(layer_in,layer_out,bias=True,is_first=idx==0))
                ## if an intermdeiate layer
                else:
                    self.net_layers.append(ResidualSineLayer(layer_in,bias=True,ave_first=idx>1,ave_second=idx==(self.n_layers-2)))
            ## if final layer
            else:
                final_linear = nn.Linear(layer_in,layer_out)
                ## initialize weights for the final layer
                with torch.no_grad():
                    final_linear.weight.uniform_(-np.sqrt(6 / (layer_in)) / self.Omega_0, np.sqrt(6 / (layer_in)) / self.Omega_0)
                self.net_layers.append(final_linear)

    def forward(self,x):
        for net_layer in self.net_layers:
            x = net_layer(x)
        return x

In [ ]:
def size_of_network(n_layers, n_neurons, d_in, d_out, is_residual = True):
    # Adding input layer
    layers = [d_in]
    # layers = [3]

    # Adding hidden layers
    layers.extend([n_neurons]*n_layers)
    # layers = [3, 5, 5, 5]

    # Adding output layer
    layers.append(d_out)
    # layers = [3, 5, 5, 5, 1]

    # Number of steps
    n_layers = len(layers)-1
    # n_layers = 5 - 1 = 4

    n_params = 0

    # np.arange(4) = [0, 1, 2, 3]
    for ndx in np.arange(n_layers):

        # number of neurons in below layer
        layer_in = layers[ndx]

        # number of neurons in above layer
        layer_out = layers[ndx+1]

        # max number of neurons in both the layer
        og_layer_in = max(layer_in,layer_out)

        # if lower layer is the input layer
        # or the upper layer is the output layer
        if ndx==0 or ndx==(n_layers-1):
            # Adding weight corresponding to every neuron for every input neuron
            # Adding bias for every neuron in the upper layer
            n_params += ((layer_in+1)*layer_out)

        else:

            # If the layer is residual then proceed as follows as there will be more weights if residual layer is included
            if is_residual:
                # doubt in the following two lines
                n_params += (layer_in*og_layer_in)+og_layer_in
                n_params += (og_layer_in*layer_out)+layer_out

            # if the layer is non residual then simply add number of weights and biases as follows
            else:
                n_params += ((layer_in+1)*layer_out)
            #
        #
    #

    return n_params

In [ ]:
def compute_PSNR(arrgt,arr_recon):
    diff = arrgt - arr_recon
    sqd_max_diff = (np.max(arrgt)-np.min(arrgt))**2
    snr = 10*np.log10(sqd_max_diff/np.mean(diff**2))
    return snr

In [ ]:
def srs(numOfPoints, valid_pts, percentage, isMaskPresent, mask_array):

    # getting total number of sampled points
    numberOfSampledPoints = int((valid_pts/100) * percentage)

    # storing corner indices in indices variable
    indices = set()

    # As long as we don't get the required amount of sample points keep finding the random numbers
    while(len(indices) < numberOfSampledPoints):
        rp = random.randint(0, numOfPoints-1)
        if isMaskPresent and mask_array[rp] == 0:
            continue
        indices.add(rp)

    # return indices
    return indices

In [ ]:
def findMultiVariatePSNR(var_name, total_vars, actual, pred):
    # print('Printing PSNR')
    tot = 0
    psnr_list = []
    for j in range(total_vars):
        psnr = compute_PSNR(actual[:,j], pred[:,j])
        psnr_list.append(psnr)
        tot += psnr
        print(var_name, ' PSNR:', psnr)
    avg_psnr = tot/total_vars
    print('\nAverage psnr : ', avg_psnr)
     #this function is calculating the psnr of final epoch (or whenever it is called) of each variable and then averaging it
     #Thus individual epochs psnr is not calculated

    return psnr_list, avg_psnr

In [ ]:
def compute_rmse(actual, predicted):
    mse = np.mean((actual - predicted) ** 2)
    return np.sqrt(mse)

def denormalizeValue(total_vars, to, ref):
    to_arr = np.array(to)
    for i in range(total_vars):
        min_data = np.min(ref[:, i])
        max_data = np.max(ref[:, i])
        to_arr[:, i] = (((to[:, i] * 0.5) + 0.5) * (max_data - min_data)) + min_data
    return to_arr

In [ ]:
def makeVTI(data, val, n_predictions, n_pts, total_vars, var_name, dim, isMaskPresent, mask_arr, vti_path, vti_name, normalizedVersion = False):
    nn_predictions = denormalizeValue(total_vars, n_predictions, val) if not normalizedVersion else n_predictions
    writer = vtkXMLImageDataWriter()
    writer.SetFileName(vti_path + vti_name)
    img = vtkImageData()
    img.CopyStructure(data)
    if not isMaskPresent:
        for i in range(total_vars):
            f = var_name[i]
            temp = nn_predictions[:, i]
            arr = vtkFloatArray()
            for j in range(n_pts):
                arr.InsertNextValue(temp[j])
            arr.SetName(f)
            img.GetPointData().AddArray(arr)
        # print(img)
        writer.SetInputData(img)
        writer.Write()
        print(f'Vti File written successfully at {vti_path}{vti_name}')
    else:
        for i in range(total_vars):
            f = var_name[i]
            temp = nn_predictions[:, i]
            idx = 0
            arr = vtkFloatArray()
            for j in range(n_pts):
                if(mask_arr[j] == 1):
                    arr.InsertNextValue(temp[idx])
                    idx += 1
                else:
                    arr.InsertNextValue(0.0)
            arr.SetName('p_' + f)
            data.GetPointData().AddArray(arr)
        # print(data)
        writer.SetInputData(data)
        writer.Write()
        print(f'Vti File written successfully at {vti_path}{vti_name}')

In [ ]:
def getImageData(actual_img, val, n_pts, var_name, isMaskPresent, mask_arr):
    img = vtkImageData()
    img.CopyStructure(actual_img)
    # if isMaskPresent:
    #     img.DeepCopy(actual_img)
    # img.SetDimensions(dim)
    # img.SetOrigin(actual_img.GetOrigin())
    # img.SetSpacing(actual_img.GetSpacing())
    if not isMaskPresent:
        f = var_name
        data = val
        arr = vtkFloatArray()
        for j in range(n_pts):
            arr.InsertNextValue(data[j])
        arr.SetName(f)
        img.GetPointData().SetScalars(arr)
    else:
        f = var_name
        data = val
        idx = 0
        arr = vtkFloatArray()
        for j in range(n_pts):
            if(mask_arr[j] == 1):
                arr.InsertNextValue(data[idx])
                idx += 1
            else:
                arr.InsertNextValue(0.0)
        arr.SetName(f)
        img.GetPointData().SetScalars(arr)
    return img

In [ ]:
from argparse import Namespace

# Parameters (simulating argparse in a Jupyter Notebook)
args = Namespace(
    n_neurons=150,
    n_layers=6,
    epochs=200,  # Required argument: Set the number of epochs
    batchsize=2048,
    lr=0.00005,
    no_decay=False,
    decay_rate=0.8,
    decay_at_interval=True,
    decay_interval=15,
    datapath='/content/Teardrop_Gaussian.vti',  # Required: Set the path to your data
    outpath='./models/',
    exp_path='../logs/',
    modified_data_path='./data/',
    dataset_name='3d_data',  # Required: Set the dataset name
    vti_name='predicted.vti',  # Required: Name of the dataset
    vti_path='./data/'
)

print(args, end='\n\n')

# Assigning parameters to variables
LR = args.lr
BATCH_SIZE = args.batchsize
decay_rate = args.decay_rate
decay_at_equal_interval = args.decay_at_interval

decay = not args.no_decay
MAX_EPOCH = args.epochs

n_neurons = args.n_neurons
n_layers = args.n_layers + 2
decay_interval = args.decay_interval
outpath = args.outpath
exp_path = args.exp_path
datapath = args.datapath
modified_data_path = args.modified_data_path
dataset_name = args.dataset_name
vti_name = args.vti_name
vti_path = args.vti_path

# Displaying the final configuration
print(f"Learning Rate: {LR}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Decay Rate: {decay_rate}")
print(f"Max Epochs: {MAX_EPOCH}")
print(f"Number of Neurons per Layer: {n_neurons}")
print(f"Number of Layers (including input/output): {n_layers}")
print(f"Data Path: {datapath}")
print(f"Output Path: {outpath}")
print(f"Dataset Name: {dataset_name}")
print(f"Vti Name: {vti_name}")

Namespace(n_neurons=150, n_layers=6, epochs=200, batchsize=2048, lr=5e-05, no_decay=False, decay_rate=0.8, decay_at_interval=True, decay_interval=15, datapath='/content/Teardrop_Gaussian.vti', outpath='./models/', exp_path='../logs/', modified_data_path='./data/', dataset_name='3d_data', vti_name='predicted.vti', vti_path='./data/')

Learning Rate: 5e-05
Batch Size: 2048
Decay Rate: 0.8
Max Epochs: 200
Number of Neurons per Layer: 150
Number of Layers (including input/output): 8
Data Path: /content/Teardrop_Gaussian.vti
Output Path: ./models/
Dataset Name: 3d_data
Vti Name: predicted.vti


In [ ]:
# Variable Initialization
var_name = []
total_vars = None  # Number of variables
univariate = None  # True if dataset has one variable, else False
group_size = 5000  # Group size during testing


# Constructing the log file name
log_file = (
    f'train_{dataset_name}_{n_layers-2}rb_{n_neurons}n_{BATCH_SIZE}bs_'
    f'{LR}lr_{decay}decay_{decay_rate}dr_'
    f'{"decayingAtInterval" + str(decay_interval) if decay_at_equal_interval else "decayingWhenLossIncr"}'
)

print(log_file)

train_3d_data_6rb_150n_2048bs_5e-05lr_Truedecay_0.8dr_decayingAtInterval15


In [ ]:
n_pts = None  # Number of points in the dataset
n_dim = None  # Dimensionality of the data
dim = None  # Other dimension-specific information

print("Decay:", decay)
print(f'Extracting variables from path: {datapath}', end="\n\n")

# Placeholder for data
data_array = []
scalar_data = None

Decay: True
Extracting variables from path: /content/Teardrop_Gaussian.vti



In [ ]:
# # Reading values from .vti files
# reader = vtk.vtkXMLImageDataReader()
# reader.SetFileName(datapath)
# reader.Update()

# data = reader.GetOutput()
# scalar_data = data
# pdata = data.GetPointData()
# n_pts = data.GetNumberOfPoints()
# dim = data.GetDimensions()
# n_dim = len(dim)
# total_arr = pdata.GetNumberOfArrays()

# print("n_pts:", n_pts, "dim:", dim, "n_dim:", n_dim, "total_arr:", total_arr)

# mask_arr = []
# valid_pts = 0
# var_name = []
# data_array = []

# # Extracting data from the .vti file
# for i in range(total_arr):
#     a_name = pdata.GetArrayName(i)
#     if a_name in ['vtkValidPointMask', 'Swirl']:
#         continue

#     cur_arr = pdata.GetArray(a_name)
#     n_components = cur_arr.GetNumberOfComponents()

#     if n_components == 1:
#         var_name.append(a_name)
#         data_array.append(vtk_to_numpy(cur_arr))
#     else:
#         component_names = [f"{a_name}_{c}" for c in ['x', 'y', 'z'][:n_components]]
#         var_name.extend(component_names)
#         for c in range(n_components):
#             c_data = [cur_arr.GetComponent(j, c) for j in range(n_pts)]
#             data_array.append(np.array(c_data))

# valid_pts = n_pts  # Assume all points are valid for simplicity
# total_vars = len(var_name)
# univariate = total_vars == 1

# # Prepare numpy arrays for coordinates and variable values
# cord = np.zeros((valid_pts, n_dim))
# val = np.zeros((valid_pts, total_vars))

# # Store data in numpy arrays
# for i in range(n_pts):
#     pt = scalar_data.GetPoint(i)
#     cord[i, :] = pt
#     val[i, :] = [arr[i] for arr in data_array]

# # Display final information
# print("Total Variables:", total_vars)
# print("Univariate:", univariate)
# print("Coordinates Shape:", cord.shape)
# print("Values Shape:", val.shape)

# Reading values from .vti files
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(datapath)
reader.Update()

data = reader.GetOutput()
scalar_data = data
pdata = data.GetPointData()
n_pts = data.GetNumberOfPoints()
dim = data.GetDimensions()
n_dim = len(dim)
total_arr = pdata.GetNumberOfArrays()

print("n_pts:", n_pts, "dim:", dim, "n_dim:", n_dim, "total_arr:", total_arr)

var_name = []
data_array = []

# Extracting data from the .vti file
for i in range(total_arr):
    a_name = pdata.GetArrayName(i)

    cur_arr = pdata.GetArray(a_name)
    n_components = cur_arr.GetNumberOfComponents()

    if n_components == 1:
        var_name.append(a_name)
        data_array.append(vtk_to_numpy(cur_arr))
    else:
        component_names = [f"{a_name}_{c}" for c in ['x', 'y', 'z'][:n_components]]
        var_name.extend(component_names)
        for c in range(n_components):
            c_data = [cur_arr.GetComponent(j, c) for j in range(n_pts)]
            data_array.append(np.array(c_data))

total_vars = len(var_name)
univariate = total_vars == 1

# Prepare numpy arrays for coordinates and variable values
cord = np.zeros((n_pts, n_dim))
val = np.zeros((n_pts, total_vars))

# Store data in numpy arrays
for i in range(n_pts):
    pt = scalar_data.GetPoint(i)
    cord[i, :] = pt
    val[i, :] = [arr[i] for arr in data_array]

# Display final information
print("Total Variables:", total_vars)
print("Univariate:", univariate)
print("Coordinates Shape:", cord.shape)
print("Values Shape:", val.shape)

n_pts: 262144 dim: (64, 64, 64) n_dim: 3 total_arr: 2
Total Variables: 2
Univariate: False
Coordinates Shape: (262144, 3)
Values Shape: (262144, 2)


In [ ]:
# # Ensure modified data path exists
# if not os.path.exists(modified_data_path):
#     os.mkdir(modified_data_path)

# Save raw coordinates and values
# np.save(f'{modified_data_path}cord.npy', cord)
# np.save(f'{modified_data_path}val.npy', val)

# # Create copies of non-normalized data
# nn_cord = cord.copy()
# nn_val = val.copy()

# === Separate Normalization for Values ===
# We assume the variable order is:
#   - Means: indices 0,1,2
#   - Std Devs: indices 3,4,5
#   - Weights: indices 6,7,8

# # We'll store normalization parameters so that we can invert normalization later.
# norm_params = {}
# epsilon = 1e-8  # to avoid log(0)

# # Normalize Means to [-1,1] using min–max normalization
# for i in range(3):
#     min_val = np.min(val[:, i])
#     max_val = np.max(val[:, i])
#     norm_params[var_name[i]] = (min_val, max_val)
#     val[:, i] = 2.0 * ((val[:, i] - min_val) / (max_val - min_val) - 0.5)

# # Normalize Std Devs: first take log, then min–max to [-1,1]
# for i in range(3, 6):
#     log_vals = np.log(val[:, i] + epsilon)
#     min_val = np.min(log_vals)
#     max_val = np.max(log_vals)
#     norm_params[var_name[i]] = (min_val, max_val)
#     val[:, i] = 2.0 * ((log_vals - min_val) / (max_val - min_val) - 0.5)

# # Normalize Weights: take log, then min–max to [-1,1]
# for i in range(6, 9):
#     log_vals = np.log(val[:, i] + epsilon)
#     min_val = np.min(log_vals)
#     max_val = np.max(log_vals)
#     norm_params[var_name[i]] = (min_val, max_val)
#     val[:, i] = 2.0 * ((log_vals - min_val) / (max_val - min_val) - 0.5)

# norm_params = {}
real_data=val.copy()
print(np.max(real_data))
# Normalize values between -1 and 1
for i in range(total_vars):
    min_data = np.min(val[:, i])
    max_data = np.max(val[:, i])
    # norm_params[var_name[i]] = (min_data, max_data)
    val[:, i] = 2.0 * ((val[:, i] - min_data) / (max_data - min_data) - 0.5)

# Normalize Coordinates to [-1,1]
for i in range(n_dim):
    # Use (dim[i]-1] so that coordinates go from 0 to dim[i]-1.
    cord[:, i] = 2.0 * (cord[:, i] / (dim[i] - 1) - 0.5)

# # Normalize coordinates between 0 and 1
# for i in range(n_dim):
#     cord[:, i] = cord[:, i] / dim[i]


# # Save normalized values and coordinates
# np.save(f'{modified_data_path}n_cord.npy', cord)
# np.save(f'{modified_data_path}n_val.npy', val)
n_cord = cord.copy()
n_val = val.copy()

# # Reload data for verification
# n_cord = np.load(f'{modified_data_path}n_cord.npy')
# n_val = np.load(f'{modified_data_path}n_val.npy')
# cord = np.load(f'{modified_data_path}cord.npy')
# val = np.load(f'{modified_data_path}val.npy')
means=n_val[:,0]
stds=n_val[:,1]
# Convert normalized data to PyTorch tensors
torch_coords = torch.from_numpy(n_cord)
torch_means = torch.from_numpy(means)
torch_stds =torch.from_numpy(stds)
# Display dataset details
print('Dataset Name:', dataset_name)
print('Total Variables:', total_vars)
print('Variables Name:', var_name, end="\n\n")
print('Total Points in Data:', n_pts)
print('Dimension of the Dataset:', dim)
print('Number of Dimensions:', n_dim)
print('Coordinate Tensor Shape:', torch_coords.shape)
print('Scalar means Values Tensor Shape:', torch_means.shape)
print('Scalar stds Values Tensor Shape:', torch_stds.shape)

print('\n###### Data setup is complete, now starting training ######\n')

161.9956817626953
Dataset Name: 3d_data
Total Variables: 2
Variables Name: ['Average', 'Standard_Deviation']

Total Points in Data: 262144
Dimension of the Dataset: (64, 64, 64)
Number of Dimensions: 3
Coordinate Tensor Shape: torch.Size([262144, 3])
Scalar means Values Tensor Shape: torch.Size([262144])
Scalar stds Values Tensor Shape: torch.Size([262144])

###### Data setup is complete, now starting training ######



In [ ]:
# Prepare the DataLoader
train_dataloader_mean = DataLoader(
    TensorDataset(torch_coords, torch_means),
    batch_size=BATCH_SIZE,
    pin_memory=True,
    shuffle=True,
    num_workers=4
)
# Model configuration
obj = {
    'total_vars': 1,
    'dim': n_dim,
    'n_neurons': n_neurons,
    'n_layers': n_layers
}

# Initialize the model, optimizer, and loss function
model_mean = MyResidualSirenNet(obj).to(device)
print(model_mean)

optimizer = optim.Adam(model_mean.parameters(), lr=LR, betas=(0.9, 0.999))
print(optimizer)

criterion = nn.MSELoss()
print(criterion)

# Training configuration summary
print('\nLearning Rate:', LR)
print('Max Epochs:', MAX_EPOCH)
print('Batch Size:', BATCH_SIZE)
print('Number of Hidden Layers:', obj['n_layers'] - 2)
print('Number of Neurons per Layer:', obj['n_neurons'])

if decay:
    print('Decay Rate:', decay_rate)
    if decay_at_equal_interval:
        print(f'Rate decays every {decay_interval} epochs.')
    else:
        print('Rate decays when the current epoch loss is greater than the previous epoch loss.')
else:
    print('No decay!')
print()

MyResidualSirenNet(
  (net_layers): ModuleList(
    (0): SineLayer(
      (linear): Linear(in_features=3, out_features=150, bias=True)
    )
    (1-6): 6 x ResidualSineLayer(
      (linear_1): Linear(in_features=150, out_features=150, bias=True)
      (linear_2): Linear(in_features=150, out_features=150, bias=True)
    )
    (7): Linear(in_features=150, out_features=1, bias=True)
  )
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 5e-05
    maximize: False
    weight_decay: 0
)
MSELoss()

Learning Rate: 5e-05
Max Epochs: 200
Batch Size: 2048
Number of Hidden Layers: 6
Number of Neurons per Layer: 150
Decay Rate: 0.8
Rate decays every 15 epochs.



/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
train_loss_list = []
best_epoch = -1
best_loss = 1e8
best_model=""
from tqdm import tqdm
# Ensure the output path exists
if not os.path.exists(outpath):
    os.makedirs(outpath)

# Training loop
for epoch in tqdm(range(MAX_EPOCH)):
    model_mean.train()
    temp_loss_list = []
    start = time.time()

    # Batch-by-batch training
    for X_train, y_train in train_dataloader_mean:
        X_train = X_train.type(torch.float32).to(device)
        y_train = y_train.type(torch.float32).to(device)

        if univariate:
            y_train = y_train.squeeze()

        optimizer.zero_grad()
        predictions = model_mean(X_train)
        predictions = predictions.squeeze()
        loss = criterion(predictions, y_train)
        loss.backward()
        optimizer.step()

        # Track batch loss
        temp_loss_list.append(loss.detach().cpu().numpy())

    # Calculate epoch loss
    epoch_loss = np.average(temp_loss_list)

    # Learning rate decay
    if decay:
        if decay_at_equal_interval:
            if epoch >= decay_interval and epoch % decay_interval == 0:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= decay_rate
        # else:
        #     if epoch > 0 and epoch_loss > train_loss_list[-1]:
        #         for param_group in optimizer.param_groups:
        #             param_group['lr'] *= decay_rate
        if epoch > 0 and epoch_loss > train_loss_list[-1]:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= decay_rate

    # Track losses and best model
    train_loss_list.append(epoch_loss)
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        best_epoch = epoch+1
        if(best_model==0):
            best_model=model_mean.state_dict()
        else:
            best_model=model_mean.state_dict()

    end = time.time()
    print(
        f"Epoch: {epoch + 1}/{MAX_EPOCH} | Train Loss: {train_loss_list[-1]} | "
        f"Time: {round(end - start, 2)}s ({device}) | LR: {optimizer.param_groups[0]['lr']}"
    )

    # Save model at intervals
    if (epoch + 1) % 50 == 0:
        model_name = (
            f'train_{dataset_name}_{epoch + 1}ep_{n_layers - 2}rb_{n_neurons}n_'
            f'{BATCH_SIZE}bs_{LR}lr_{decay}decay_{decay_rate}dr_'
            f'{"decayingAtInterval" + str(decay_interval)+"mean" if decay_at_equal_interval else "decayingWhenLossIncr"}'
        )
        torch.save(
            {"epoch": epoch + 1, "model_state_dict": model_mean.state_dict()},
            os.path.join(outpath, f'{model_name}_mean.pth')
        )

# Final summary
print('\nEpoch with Least Loss:', best_epoch, '| Loss:', best_loss, '\n')

# Save the final model
model_name = f'siren_compressor'
torch.save(
    {"epoch": MAX_EPOCH, "model_state_dict": model_mean.state_dict()},
    os.path.join(outpath, f'{model_name}_mean.pth')
)
torch.save(
    {"epoch": best_epoch, "model_state_dict": best_model},
    os.path.join(outpath, f'{best_epoch}_mean.pth')
)


  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  0%|          | 0/200 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Initialize prediction lists
prediction_list = [[] for _ in range(1)]
total_vars=1
# Inference loop
model_mean = MyResidualSirenNet(obj).to(device)
state_dict = torch.load('/content/150_neurons_mean.pth')['model_state_dict']
model_mean.load_state_dict(state_dict)
with torch.no_grad():
    for i in range(0, torch_coords.shape[0], group_size):
        coords = torch_coords[i:min(i + group_size, torch_coords.shape[0])].type(torch.float32).to(device)
        vals = model_mean(coords)
        vals = vals.to('cpu')

        for j in range(total_vars):
            prediction_list[j].append(vals[:, j])

# Extract and concatenate predictions
extracted_list = [[] for _ in range(1)]
for i in range(len(prediction_list[0])):
    for j in range(1):
        el = prediction_list[j][i].detach().numpy()
        extracted_list[j].append(el)

for j in range(1):
    extracted_list[j] = np.concatenate(extracted_list[j], dtype='float32')

# Final prediction (normalized)
n_predictions_means = np.array(extracted_list).T

# Compute PSNR
#findMultiVariatePSNR(var_name[0], total_vars, n_val[:,0], n_predictions_means[:,0])
print("mean",compute_PSNR(n_val[:,0],n_predictions_means[:,0]))
# Compute RMSE
rmse = compute_rmse(n_val[:,0], n_predictions_means[:,0])
print("RMSE:", rmse)

mean 72.7810994165529
RMSE: 0.00045917160629419825


#finetunning model

In [ ]:
import numpy as np
import torch
from torch import nn, optim
import torch.optim as optim
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import os
import time
from tqdm import tqdm # For progress bar

# Re-define your model classes (SineLayer, ResidualSineLayer, MyResidualSirenNet)
# These are essential for re-building the pruned model.
# (Copy and paste them from your notebook)

class SineLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True, is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features,
                                             1 / self.in_features)
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0,
                                             np.sqrt(6 / self.in_features) / self.omega_0)

    def forward(self, x):
        x = self.linear(x)
        return torch.sin(self.omega_0 * x)

class ResidualSineLayer(nn.Module):
    def __init__(self, features, bias=True, ave_first=False, ave_second=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.features = features
        self.linear_1 = nn.Linear(features, features, bias=bias)
        self.linear_2 = nn.Linear(features, features, bias=bias)
        self.weight_1 = .5 if ave_first else 1
        self.weight_2 = .5 if ave_second else 1
        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            self.linear_1.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                           np.sqrt(6 / self.features) / self.omega_0)
            self.linear_2.weight.uniform_(-np.sqrt(6 / self.features) / self.omega_0,
                                           np.sqrt(6 / self.features) / self.omega_0)

    def forward(self, input):
        linear_1_out = self.linear_1(self.weight_1*input)
        sine_1_out = torch.sin(self.omega_0 * linear_1_out)
        sine_2_out = torch.sin(self.omega_0 * self.linear_2(sine_1_out))
        return self.weight_2*(input+sine_2_out)

class MyResidualSirenNet(nn.Module):
    def __init__(self, obj):
        super(MyResidualSirenNet, self).__init__()
        self.Omega_0 = 30
        self.n_layers = obj['n_layers']
        self.input_dim = obj['dim']
        self.output_dim = obj['total_vars']
        self.neurons_per_layer = obj['n_neurons']

        self.layers_dim_list = [self.input_dim]
        for i in range(self.n_layers - 1):
            self.layers_dim_list.append(self.neurons_per_layer)
        self.layers_dim_list.append(self.output_dim)

        self.net_layers = nn.ModuleList()
        for idx in np.arange(self.n_layers):
            layer_in = self.layers_dim_list[idx]
            layer_out = self.layers_dim_list[idx + 1]

            if idx == 0:
                self.net_layers.append(SineLayer(layer_in, layer_out, bias=True, is_first=True))
            elif idx != self.n_layers - 1:
                self.net_layers.append(ResidualSineLayer(layer_in, bias=True, ave_first=idx > 1, ave_second=idx == (self.n_layers - 2)))
            else:
                final_linear = nn.Linear(layer_in, layer_out)
                with torch.no_grad():
                    final_linear.weight.uniform_(-np.sqrt(6 / (layer_in)) / self.Omega_0, np.sqrt(6 / (layer_in)) / self.Omega_0)
                self.net_layers.append(final_linear)

    def forward(self, x):
        for net_layer in self.net_layers:
            x = net_layer(x)
        return x

# Your utility functions (compute_PSNR, denormalizeValue, etc.)
def compute_PSNR(arrgt,arr_recon):
    diff = arrgt - arr_recon
    sqd_max_diff = (np.max(arrgt)-np.min(arrgt))**2
    # Ensure MSE is not zero for log calculation
    mse = np.mean(diff**2)
    if mse == 0:
        return float('inf')
    psnr = 10*np.log10(sqd_max_diff/mse)
    return psnr

def compute_rmse(actual, predicted):
    mse = np.mean((actual - predicted) ** 2)
    return np.sqrt(mse)

def denormalizeValue(total_vars, to, ref):
    to_arr = np.array(to)
    for i in range(total_vars):
        min_data = np.min(ref[:, i])
        max_data = np.max(ref[:, i])
        # Invert the normalization: val[:, i] = 2.0 * ((val[:, i] - min_data) / (max_data - min_data) - 0.5);
        # val_norm = 2 * (val_orig - min_data) / (max_data - min_data) - 1
        # (val_norm + 1) / 2 = (val_orig - min_data) / (max_data - min_data)
        # val_orig = ((val_norm + 1) / 2) * (max_data - min_data) + min_data
        to_arr[:, i] = (((to[:, i] * 0.5) + 0.5) * (max_data - min_data)) + min_data
    return to_arr

# Assume args, n_dim, total_vars, var_name, dim, n_pts, real_data, n_val, torch_coords, torch_means, torch_stds are loaded as in your notebook
# You will need to load your data and initial models first.
# This part is critical for loading the actual data and original model states for pruning and evaluation.
# Given your notebook, the PSNR for mean model is `72.7810994165529`
# PSNR for std model is `61.882343845798715`
# You want to prune the model while maintaining a PSNR of at least 70.0.

# Mocking your data loading and setup based on your notebook
from argparse import Namespace
args = Namespace(
    n_neurons=150,
    n_layers=6, # This means 6 residual blocks + input + output = 8 layers in total
    epochs=200,
    batchsize=512,
    lr=0.00005,
    no_decay=False,
    decay_rate=0.8,
    decay_at_interval=True,
    decay_interval=15,
    datapath='/content/Teardrop_Gaussian.vti',
    outpath='./models/',
    exp_path='../logs/',
    modified_data_path='./data/',
    dataset_name='3d_data',
    vti_name='predicted.vti',
    vti_path='./data/'
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# --- Dummy data loading for demonstration. Replace with your actual VTK loading ---
# For actual execution, you'd run the VTK loading cells from your notebook.
# This is to make the pruning code runnable independently if you don't have VTK setup directly.
n_pts = 262144
n_dim = 3
dim = (64, 64, 64)
total_vars = 2
var_name = ['Average', 'Standard_Deviation']

# Create dummy coordinate and value arrays for demonstration # Unnormalized original data

# Normalize values between -1 and 1
n_val = real_data.copy()
for i in range(total_vars):
    min_data = np.min(n_val[:, i])
    max_data = np.max(n_val[:, i])
    n_val[:, i] = 2.0 * ((n_val[:, i] - min_data) / (max_data - min_data) - 0.5)

# Normalize Coordinates to [-1,1]
# n_cord = cord.copy()
# for i in range(n_dim):
#     n_cord[:, i] = 2.0 * (n_cord[:, i] / (dim[i] - 1) - 0.5)
# n_cord=
torch_coords = torch.from_numpy(n_cord).float()
torch_means = torch.from_numpy(n_val[:, 0]).float()
torch_stds = torch.from_numpy(n_val[:, 1]).float()

# --- End of dummy data loading ---

# Model configuration from args
obj_original = {
    'total_vars': 1,
    'dim': n_dim,
    'n_neurons': args.n_neurons,
    'n_layers': args.n_layers + 2 # Includes input and output as per your notebook's definition
}

# Load the trained mean model
original_mean_model = MyResidualSirenNet(obj_original).to(device)
model_path = os.path.join(args.outpath, '/content/150_neurons_mean.pth') # Assuming you saved your best mean model as 200_mean.pth or similar
if os.path.exists(model_path):
    print(f"Loading original model from {model_path}")
    original_state_dict = torch.load(model_path, map_location=device)['model_state_dict']
    original_mean_model.load_state_dict(original_state_dict)
else:
    print(f"Warning: Model file not found at {model_path}. Please ensure your trained model is saved correctly.")
    # As a fallback for demonstration, initialize random weights if model is not found
    print("Initializing a new model for demonstration purposes.")
    # You would typically train it here or load a different existing one.

original_mean_model.eval()

# Calculate initial PSNR for verification
with torch.no_grad():
    test_coords = torch_coords.to(device)
    original_predictions = original_mean_model(test_coords).squeeze().cpu().numpy()
initial_psnr = compute_PSNR(n_val[:, 0], original_predictions)
print(f"Initial PSNR (Mean Model): {initial_psnr:.4f}")
print(f"Original model parameters: {sum(p.numel() for p in original_mean_model.parameters())}")

# --- PRUNING FUNCTION ---
def prune_siren_model(original_model, original_obj, prune_ratio, train_dataloader, original_n_val_target, fine_tune_epochs=50, fine_tune_lr=0.00001, target_psnr=70.0):
    """
    Performs structured pruning on the SIREN model by removing neurons.

    Args:
        original_model (nn.Module): The already trained MyResidualSirenNet model.
        original_obj (dict): The original configuration object for MyResidualSirenNet.
        prune_ratio (float): The fraction of neurons to prune (e.g., 0.1 for 10%).
        train_dataloader (DataLoader): DataLoader for fine-tuning.
        original_n_val_target (np.ndarray): The normalized ground truth values for PSNR calculation.
        fine_tune_epochs (int): Number of epochs for fine-tuning after pruning.
        fine_tune_lr (float): Learning rate for fine-tuning.
        target_psnr (float): The minimum acceptable PSNR after pruning.

    Returns:
        tuple: (pruned_model, new_n_neurons, final_psnr) if successful, else (None, None, None)
    """
    device = next(original_model.parameters()).device # Get current device

    # Identify prune-able layers and collect their weights/biases
    layers_to_prune = [] # Stores (layer_idx, module_name, linear_module, is_residual_input, current_in_features)

    # First layer (SineLayer)
    # Prune its 'out_features'
    first_layer_module = original_model.net_layers[0].linear
    layers_to_prune.append((0, 'input_sine_layer', first_layer_module, False, first_layer_module.in_features))

    # ResidualSineLayers (each has linear_1 and linear_2)
    # Prune 'features' (which is both in_features and out_features for linear_1 and linear_2)
    for i in range(1, original_obj['n_layers'] - 1): # Iterate through ResidualSineLayers
        res_block_module = original_model.net_layers[i]
        # linear_1's output neurons are candidates
        layers_to_prune.append((i, f'residual_block_{i}_linear_1', res_block_module.linear_1, False, res_block_module.linear_1.in_features))
        # linear_2's output neurons are candidates
        layers_to_prune.append((i, f'residual_block_{i}_linear_2', res_block_module.linear_2, False, res_block_module.linear_2.in_features))

    # Last layer (Linear) - its 'in_features' are the 'out_features' of the last residual block
    # We don't prune the output layer's *output* (total_vars), only its *input* which comes from the previous layer
    # So, the effective pruning is done on the last ResidualSineLayer's output
    # This is handled implicitly by adjusting `new_n_neurons`.

    # Calculate L1 norms for all potential neurons to prune (output features)
    neuron_l1_norms = []
    # Store (l1_norm, layer_index_in_net_layers, internal_linear_idx_in_block, neuron_index)
    # We collect all neurons from intermediate layers to prune
    for layer_idx, _, linear_module, _, _ in layers_to_prune:
        # L1 norm for each *output neuron* (column of the weight matrix)
        # weight.data shape is (out_features, in_features)
        # We want to prune out_features (rows), so compute norm along dim=1 (input features)
        l1_norms = torch.norm(linear_module.weight.data, p=1, dim=1) # L1 norm per output feature
        for neuron_idx, norm_val in enumerate(l1_norms):
            neuron_l1_norms.append((norm_val.item(), layer_idx, neuron_idx))

    # Sort neurons by L1 norm (ascending)
    neuron_l1_norms.sort(key=lambda x: x[0])

    # Determine how many neurons to prune from the hidden layers (neurons_per_layer)
    # We apply the prune_ratio to the 'neurons_per_layer'
    original_hidden_neurons = original_obj['n_neurons']
    num_neurons_to_prune = int(original_hidden_neurons * prune_ratio)
    if num_neurons_to_prune < 1 and prune_ratio > 0: # Ensure at least one neuron is pruned if ratio is positive
        num_neurons_to_prune = 1
    if num_neurons_to_prune >= original_hidden_neurons: # Don't prune all
        print(f"Warning: Prune ratio {prune_ratio} is too high. Cannot prune all neurons. Adjusting to prune {original_hidden_neurons - 1} neurons.")
        num_neurons_to_prune = original_hidden_neurons - 1
    if num_neurons_to_prune < 0: # No negative pruning
        num_neurons_to_prune = 0


    # Create a set of indices of neurons to keep for each relevant layer
    # We will identify the lowest L1 norm neurons to prune.
    neurons_to_prune_per_layer_map = {} # {layer_idx: set(neuron_indices_to_prune)}

    # Collect the indices of neurons to prune based on the sorted L1 norms
    # We prune from the most "dense" layers, which are the main hidden layers (residual blocks)
    # This assumes uniform pruning across all hidden layers to maintain the residual structure's dimensions
    # For a simple uniform prune across all hidden layers:
    pruned_neurons_indices = set()
    for _, layer_idx, neuron_idx in neuron_l1_norms:
        # Apply pruning primarily to the hidden layers (layers with n_neurons)
        # The input layer's output (layer 0) and residual layers (1 to n_layers-2) all have 'n_neurons' as output features
        if original_obj['n_layers'] > 2 and layer_idx > 0 and layer_idx < original_obj['n_layers'] - 1 : # All hidden layers
            if len(pruned_neurons_indices) < num_neurons_to_prune:
                pruned_neurons_indices.add(neuron_idx) # Neuron index within its layer
            else:
                break

    # If we collected enough neurons from a variety of layers, convert set to list and sort
    if len(pruned_neurons_indices) > 0:
        pruned_neurons_indices = sorted(list(pruned_neurons_indices))
        kept_neurons_indices = sorted([i for i in range(original_hidden_neurons) if i not in pruned_neurons_indices])
    else: # No neurons pruned
        kept_neurons_indices = list(range(original_hidden_neurons))

    new_n_neurons = len(kept_neurons_indices)
    if new_n_neurons == 0:
        print("Error: Cannot prune all neurons. Resulting model would have 0 neurons.")
        return None, None, None

    print(f"Original hidden neurons: {original_hidden_neurons}, New hidden neurons: {new_n_neurons} (Pruning {num_neurons_to_prune} neurons)")


    # Construct the new, pruned model architecture
    new_obj = original_obj.copy()
    new_obj['n_neurons'] = new_n_neurons

    pruned_model = MyResidualSirenNet(new_obj).to(device)
    print(f"New pruned model architecture:\n{pruned_model}")
    print(f"New pruned model parameters: {sum(p.numel() for p in pruned_model.parameters())}")
    print(f"Parameter reduction: {1 - sum(p.numel() for p in pruned_model.parameters()) / sum(p.numel() for p in original_model.parameters()):.2%}")


    # Copy weights from original model to pruned model
    with torch.no_grad():
        # First SineLayer: Only its 'out_features' are affected
        # original_model.net_layers[0].linear.weight shape: (out_features, in_features)
        # kept_neurons_indices are for its out_features
        pruned_model.net_layers[0].linear.weight.data = original_model.net_layers[0].linear.weight.data[kept_neurons_indices, :]
        pruned_model.net_layers[0].linear.bias.data = original_model.net_layers[0].linear.bias.data[kept_neurons_indices]

        # ResidualSineLayers: These are tricky because of the skip connection and two linear layers
        # Each ResidualSineLayer(features) means both linear_1 and linear_2 have (features, features)
        # When you prune 'features', it affects both in and out dimensions of these internal layers
        # The skip connection also remains of size 'features'
        for i in range(1, original_obj['n_layers'] - 1):
            original_res_block = original_model.net_layers[i]
            pruned_res_block = pruned_model.net_layers[i]

            # linear_1: its in_features is new_n_neurons (from previous layer), its out_features is also new_n_neurons
            # original_res_block.linear_1.weight shape: (old_out_features, old_in_features)
            # You want to select rows (output features) by `kept_neurons_indices`
            # and columns (input features) by `kept_neurons_indices`
            pruned_res_block.linear_1.weight.data = original_res_block.linear_1.weight.data[kept_neurons_indices, :]
            pruned_res_block.linear_1.weight.data = pruned_res_block.linear_1.weight.data[:, kept_neurons_indices]
            pruned_res_block.linear_1.bias.data = original_res_block.linear_1.bias.data[kept_neurons_indices]

            # linear_2: same as linear_1
            pruned_res_block.linear_2.weight.data = original_res_block.linear_2.weight.data[kept_neurons_indices, :]
            pruned_res_block.linear_2.weight.data = pruned_res_block.linear_2.weight.data[:, kept_neurons_indices]
            pruned_res_block.linear_2.bias.data = original_res_block.linear_2.bias.data[kept_neurons_indices]

        # Final Linear Layer: Its 'in_features' are from the last residual block's output
        # So its columns (input features) need to be subsetted by kept_neurons_indices
        pruned_model.net_layers[-1].weight.data = original_model.net_layers[-1].weight.data[:, kept_neurons_indices]
        pruned_model.net_layers[-1].bias.data = original_model.net_layers[-1].bias.data


    # Fine-tuning
    print(f"\n--- Fine-tuning pruned model for {fine_tune_epochs} epochs ---")
    pruned_model.train()
    optimizer_pruned = optim.Adam(pruned_model.parameters(), lr=fine_tune_lr, betas=(0.9, 0.999))
    criterion = nn.MSELoss()

    best_pruned_loss = float('inf')
    best_pruned_state = None

    for epoch in tqdm(range(fine_tune_epochs)):
        epoch_loss_list = []
        for X_train, y_train in train_dataloader:
            X_train = X_train.type(torch.float32).to(device)
            y_train = y_train.type(torch.float32).to(device)
            y_train = y_train.squeeze() # Assuming your target is 1D

            optimizer_pruned.zero_grad()
            predictions = pruned_model(X_train).squeeze()
            loss = criterion(predictions, y_train)
            loss.backward()
            optimizer_pruned.step()
            epoch_loss_list.append(loss.detach().cpu().numpy())

        current_epoch_loss = np.mean(epoch_loss_list)
        if current_epoch_loss < best_pruned_loss:
            best_pruned_loss = current_epoch_loss
            best_pruned_state = pruned_model.state_dict()

        print(f"Fine-tune Epoch {epoch+1}/{fine_tune_epochs}, Loss: {current_epoch_loss:.6e}, LR: {optimizer_pruned.param_groups[0]['lr']:.2e}")

    if best_pruned_state:
        pruned_model.load_state_dict(best_pruned_state)
    pruned_model.eval()

    # Evaluate PSNR after fine-tuning
    with torch.no_grad():
        test_coords = torch_coords.to(device)
        final_predictions = pruned_model(test_coords).squeeze().cpu().numpy()
    final_psnr = compute_PSNR(original_n_val_target, final_predictions)
    print(f"\nFinal PSNR after pruning and fine-tuning: {final_psnr:.4f}")

    if final_psnr >= target_psnr:
        print(f"Pruning successful! Achieved {final_psnr:.2f} PSNR with {new_n_neurons} hidden neurons.")
        return pruned_model, new_n_neurons, final_psnr
    else:
        print(f"Pruning resulted in PSNR {final_psnr:.2f} which is below target {target_psnr:.2f}. Consider less aggressive pruning.")
        return None, None, None

# --- Main Pruning Execution ---

# Set up DataLoader for the mean model's data
# You need to make sure torch_coords and torch_means are loaded from your actual data.
train_dataloader_mean = DataLoader(
    TensorDataset(torch_coords, torch_means),
    batch_size=args.batchsize,
    pin_memory=True,
    shuffle=True,
    num_workers=0 # Set to 0 for simpler debugging in notebooks, adjust as needed
)

# Iterative pruning loop
prune_ratios = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30] # Try different pruning percentages
current_best_pruned_model = None
current_best_psnr = initial_psnr
current_n_neurons = obj_original['n_neurons']

print("\n--- Starting Iterative Pruning ---")

for p_ratio in prune_ratios:
    print(f"\nAttempting to prune with ratio: {p_ratio * 100:.0f}%")

    # Pass the *original* obj for architecture definition, but the pruning function will modify it
    pruned_model, new_neurons, final_psnr = prune_siren_model(
        original_mean_model, # The full model from previous iteration or initial model
        obj_original,   # Use the original model's parameters for structure definition
        prune_ratio=p_ratio,
        train_dataloader=train_dataloader_mean,
        original_n_val_target=n_val[:, 0], # Target for PSNR
        fine_tune_epochs=50,
        fine_tune_lr=args.lr * 0.1, # Typically lower learning rate for fine-tuning
        target_psnr=70.0
    )

    if pruned_model is not None:
        if final_psnr > current_best_psnr: # Prioritize PSNR
             # If this pruning yields a better PSNR (though we want <=70), or is first good result
            current_best_pruned_model = pruned_model
            current_best_psnr = final_psnr
            current_n_neurons = new_neurons
            print(f"Updated best model with {new_neurons} neurons and PSNR: {final_psnr:.2f}")
            # Save this best model
            model_save_name = f'pruned_siren_mean_{new_neurons}n.pth'
            torch.save(current_best_pruned_model.state_dict(), os.path.join(args.outpath, model_save_name))
            print(f"Saved best pruned model to {os.path.join(args.outpath, model_save_name)}")
        elif final_psnr >= 70.0 and sum(p.numel() for p in pruned_model.parameters()) < sum(p.numel() for p in (current_best_pruned_model if current_best_pruned_model else original_model).parameters()):
            # If PSNR is acceptable AND model is smaller than current best acceptable model
            current_best_pruned_model = pruned_model
            current_best_psnr = final_psnr
            current_n_neurons = new_neurons
            print(f"Updated best model with {new_neurons} neurons and PSNR: {final_psnr:.2f}")
            # Save this best model
            model_save_name = f'pruned_siren_mean_{new_neurons}n.pth'
            torch.save(current_best_pruned_model.state_dict(), os.path.join(args.outpath, model_save_name))
            print(f"Saved best pruned model to {os.path.join(args.outpath, model_save_name)}")
    else:
        print(f"Pruning with ratio {p_ratio * 100:.0f}% failed to meet PSNR target.")
        # If a pruning ratio fails, you might want to stop increasing the ratio or try smaller increments.
        # break # Option to stop if performance drops significantly.

print("\n--- Pruning Process Complete ---")
if current_best_pruned_model:
    print(f"Best pruned model found with {current_n_neurons} neurons and PSNR: {current_best_psnr:.2f}")
    print(f"Original model parameters: {sum(p.numel() for p in original_mean_model.parameters())}")
    print(f"Final pruned model parameters: {sum(p.numel() for p in current_best_pruned_model.parameters())}")
    print(f"Total parameter reduction: {1 - sum(p.numel() for p in current_best_pruned_model.parameters()) / sum(p.numel() for p in original_mean_model.parameters()):.2%}")
else:
    print("No pruned model found that met the PSNR target.")

Loading original model from /content/150_neurons_mean.pth
Initial PSNR (Mean Model): 72.7811
Original model parameters: 272551

--- Starting Iterative Pruning ---

Attempting to prune with ratio: 5%
Original hidden neurons: 150, New hidden neurons: 143 (Pruning 7 neurons)
New pruned model architecture:
MyResidualSirenNet(
  (net_layers): ModuleList(
    (0): SineLayer(
      (linear): Linear(in_features=3, out_features=143, bias=True)
    )
    (1-6): 6 x ResidualSineLayer(
      (linear_1): Linear(in_features=143, out_features=143, bias=True)
      (linear_2): Linear(in_features=143, out_features=143, bias=True)
    )
    (7): Linear(in_features=143, out_features=1, bias=True)
  )
)
New pruned model parameters: 247820
Parameter reduction: 9.07%

--- Fine-tuning pruned model for 50 epochs ---


  2%|▏         | 1/50 [00:04<03:18,  4.05s/it]

Fine-tune Epoch 1/50, Loss: 3.497561e-04, LR: 5.00e-06


  4%|▍         | 2/50 [00:08<03:37,  4.54s/it]

Fine-tune Epoch 2/50, Loss: 3.140939e-05, LR: 5.00e-06


...

Fine-tune Epoch 48/50, Loss: 3.425987e-06, LR: 5.00e-06


 98%|█████████▊| 49/50 [03:33<00:04,  4.36s/it]

Fine-tune Epoch 49/50, Loss: 3.119778e-06, LR: 5.00e-06


100%|██████████| 50/50 [03:38<00:00,  4.36s/it]

Fine-tune Epoch 50/50, Loss: 3.597569e-06, LR: 5.00e-06

Final PSNR after pruning and fine-tuning: 62.0631
Pruning resulted in PSNR 62.06 which is below target 70.00. Consider less aggressive pruning.
Pruning with ratio 30% failed to meet PSNR target.

--- Pruning Process Complete ---
No pruned model found that met the PSNR target.


#stds predction and trainingn

In [ ]:
# Prepare the DataLoader
train_dataloader_std= DataLoader(
    TensorDataset(torch_coords, torch_stds),
    batch_size=BATCH_SIZE,
    pin_memory=True,
    shuffle=True,
    num_workers=4
)
# Model configuration
obj = {
    'total_vars': 1,
    'dim': n_dim,
    'n_neurons': n_neurons,
    'n_layers': n_layers
}

# Initialize the model, optimizer, and loss function
model_std = MyResidualSirenNet(obj).to(device)
print(model_mean)

optimizer_std = optim.Adam(model_mean.parameters(), lr=LR, betas=(0.9, 0.999))
print(optimizer)

criterion_std = nn.MSELoss()
print(criterion_std)

# Training configuration summary
print('\nLearning Rate:', LR)
print('Max Epochs:', MAX_EPOCH)
print('Batch Size:', BATCH_SIZE)
print('Number of Hidden Layers:', obj['n_layers'] - 2)
print('Number of Neurons per Layer:', obj['n_neurons'])

if decay:
    print('Decay Rate:', decay_rate)
    if decay_at_equal_interval:
        print(f'Rate decays every {decay_interval} epochs.')
    else:
        print('Rate decays when the current epoch loss is greater than the previous epoch loss.')
else:
    print('No decay!')
print()

MyResidualSirenNet(
  (net_layers): ModuleList(
    (0): SineLayer(
      (linear): Linear(in_features=3, out_features=150, bias=True)
    )
    (1-6): 6 x ResidualSineLayer(
      (linear_1): Linear(in_features=150, out_features=150, bias=True)
      (linear_2): Linear(in_features=150, out_features=150, bias=True)
    )
    (7): Linear(in_features=150, out_features=1, bias=True)
  )
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 5e-05
    maximize: False
    weight_decay: 0
)
MSELoss()

Learning Rate: 5e-05
Max Epochs: 200
Batch Size: 512
Number of Hidden Layers: 6
Number of Neurons per Layer: 150
Decay Rate: 0.8
Rate decays every 15 epochs.



/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
train_loss_list = []
best_epoch = -1
best_loss = 1e8
best_model=""
from tqdm import tqdm
# Ensure the output path exists
if not os.path.exists(outpath):
    os.makedirs(outpath)

# Training loop
for epoch in tqdm(range(MAX_EPOCH)):
    model_std.train()
    temp_loss_list = []
    start = time.time()

    # Batch-by-batch training
    for X_train, y_train in train_dataloader_std:
        X_train = X_train.type(torch.float32).to(device)
        y_train = y_train.type(torch.float32).to(device)

        if univariate:
            y_train = y_train.squeeze()

        optimizer_std.zero_grad()
        predictions = model_mean(X_train)
        predictions = predictions.squeeze()
        loss = criterion_std(predictions, y_train)
        loss.backward()
        optimizer.step()

        # Track batch loss
        temp_loss_list.append(loss.detach().cpu().numpy())

    # Calculate epoch loss
    epoch_loss = np.average(temp_loss_list)

    # Learning rate decay
    if decay:
        if decay_at_equal_interval:
            if epoch >= decay_interval and epoch % decay_interval == 0:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= decay_rate
        # else:
        #     if epoch > 0 and epoch_loss > train_loss_list[-1]:
        #         for param_group in optimizer.param_groups:
        #             param_group['lr'] *= decay_rate
        if epoch > 0 and epoch_loss > train_loss_list[-1]:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= decay_rate

    # Track losses and best model
    train_loss_list.append(epoch_loss)
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        best_epoch = epoch+1
        if(best_model==0):
            best_model=model_mean.state_dict()
        else:
            best_model=model_mean.state_dict()

    end = time.time()
    print(
        f"Epoch: {epoch + 1}/{MAX_EPOCH} | Train Loss: {train_loss_list[-1]} | "
        f"Time: {round(end - start, 2)}s ({device}) | LR: {optimizer.param_groups[0]['lr']}"
    )

    # Save model at intervals
    if (epoch + 1) % 50 == 0:
        model_name = (
            f'train_{dataset_name}_{epoch + 1}ep_{n_layers - 2}rb_{n_neurons}n_'
            f'{BATCH_SIZE}bs_{LR}lr_{decay}decay_{decay_rate}dr_'
            f'{"decayingAtInterval" + str(decay_interval)+"std" if decay_at_equal_interval else "decayingWhenLossIncr"}'
        )
        torch.save(
            {"epoch": epoch + 1, "model_state_dict": model_mean.state_dict()},
            os.path.join(outpath, f'{model_name}_std.pth')
        )

# Final summary
print('\nEpoch with Least Loss:', best_epoch, '| Loss:', best_loss, '\n')

# Save the final model
model_name = f'siren_compressor'
torch.save(
    {"epoch": MAX_EPOCH, "model_state_dict": model_mean.state_dict()},
    os.path.join(outpath, f'{model_name}_std.pth')
)
torch.save(
    {"epoch": best_epoch, "model_state_dict": best_model},
    os.path.join(outpath, f'{best_epoch}_std.pth')
)


  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  0%|          | 1/200 [00:07<24:17,  7.32s/it]

Epoch: 1/200 | Train Loss: 0.009194845333695412 | Time: 7.32s (cuda) | LR: 5e-05


  1%|          | 2/200 [00:13<22:01,  6.67s/it]

Epoch: 2/200 | Train Loss: 0.0028720130212605 | Time: 6.22s (cuda) | LR: 5e-05


  2%|▏         | 3/200 [00:20<22:06,  6.74s/it]

...

Epoch: 198/200 | Train Loss: 3.0763708309677895e-06 | Time: 5.32s (cuda) | LR: 2.9514790517935326e-07


100%|█████████▉| 199/200 [19:57<00:05,  6.00s/it]

Epoch: 199/200 | Train Loss: 3.0682040232932195e-06 | Time: 6.56s (cuda) | LR: 2.9514790517935326e-07


100%|██████████| 200/200 [20:02<00:00,  6.01s/it]

Epoch: 200/200 | Train Loss: 3.056057266803691e-06 | Time: 5.36s (cuda) | LR: 2.9514790517935326e-07

Epoch with Least Loss: 200 | Loss: 3.0560573e-06 



In [ ]:
# Initialize prediction lists
# Initialize prediction lists
prediction_list = [[] for _ in range(1)]
total_vars=1
# Inference loop
model_mean = MyResidualSirenNet(obj).to(device)
state_dict = torch.load(os.path.join(outpath, '200_std.pth'))['model_state_dict']
model_mean.load_state_dict(state_dict)
with torch.no_grad():
    for i in range(0, torch_coords.shape[0], group_size):
        coords = torch_coords[i:min(i + group_size, torch_coords.shape[0])].type(torch.float32).to(device)
        vals = model_mean(coords)
        vals = vals.to('cpu')

        for j in range(total_vars):
            prediction_list[j].append(vals[:, j])

# Extract and concatenate predictions
extracted_list = [[] for _ in range(1)]
for i in range(len(prediction_list[0])):
    for j in range(1):
        el = prediction_list[j][i].detach().numpy()
        extracted_list[j].append(el)

for j in range(1):
    extracted_list[j] = np.concatenate(extracted_list[j], dtype='float32')

# Final prediction (normalized)
n_predictions_stds = np.array(extracted_list).T
print(n_predictions_stds.shape)
# Compute PSNR
#findMultiVariatePSNR(var_name[0], total_vars, n_val[:,0], n_predictions_means[:,0])
print("std",compute_PSNR(n_val[:,1],n_predictions_stds[:,0]))
# Compute RMSE
rmse = compute_rmse(n_val[:,1], n_predictions_stds[:,0])
print("RMSE:", rmse)

(262144, 1)
std 61.882343845798715
RMSE: 0.001610322285992644


In [ ]:
n_predictions = np.concatenate([n_predictions_means, n_predictions_stds], axis=1)
# !rm -rf /kaggle/working/*

In [ ]:
print(os.path.getsize('/kaggle/working/models/train_3d_data_200ep_6rb_320n_512bs_5e-05lr_Truedecay_0.8dr_decayingAtInterval15.pth') / (1024 ** 2), 'MB')

4.731752395629883 MB


In [ ]:
# # vti saving path
vti_path = args.vti_path
if not os.path.exists(vti_path):
    os.makedirs(vti_path)
# vti name
vti_name = args.vti_name
isMaskPresent = False
mask_arr = []
total_vars=2
makeVTI(data,real_data, n_predictions, n_pts, total_vars, var_name, dim, isMaskPresent, mask_arr, vti_path, vti_name)

Vti File written successfully at ./data/predicted.vti
